![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [95]:
# Importing packages
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

1.1 Reading in the data

In [96]:
df = pd.read_csv("bank_marketing.csv")

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [97]:
print(df.describe())

         client_id          age           day  contact_duration  \
count  41188.00000  41188.00000  41188.000000      41188.000000   
mean   20593.50000     40.02406     15.730334        258.285010   
std    11890.09578     10.42125      8.889867        259.279249   
min        0.00000     17.00000      1.000000          0.000000   
25%    10296.75000     32.00000      8.000000        102.000000   
50%    20593.50000     38.00000     16.000000        180.000000   
75%    30890.25000     47.00000     23.000000        319.000000   
max    41187.00000     98.00000     31.000000       4918.000000   

       number_contacts  previous_campaign_contacts  cons_price_idx  \
count     41188.000000                41188.000000    41188.000000   
mean          2.567593                    0.172963       93.575664   
std           2.770014                    0.494901        0.578840   
min           1.000000                    0.000000       92.201000   
25%           1.000000                    0.00

In [98]:
for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


1.2 splitting the data

1.2.1 client df

In [99]:
# client df
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]

print(client.head())

   client_id  age        job  marital    education credit_default mortgage
0          0   56  housemaid  married     basic.4y             no       no
1          1   57   services  married  high.school        unknown       no
2          2   37   services  married  high.school             no      yes
3          3   40     admin.  married     basic.6y             no       no
4          4   56   services  married  high.school             no       no


1.2.2 campaign df

In [100]:
# campaign df
campaign = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']]

print(campaign.head())

   client_id  number_contacts  contact_duration  previous_campaign_contacts  \
0          0                1               261                           0   
1          1                1               149                           0   
2          2                1               226                           0   
3          3                1               151                           0   
4          4                1               307                           0   

  previous_outcome campaign_outcome  
0      nonexistent               no  
1      nonexistent               no  
2      nonexistent               no  
3      nonexistent               no  
4      nonexistent               no  


1.2.3 economics df

In [101]:
# economics df
economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']]

print(economics.head())

   client_id  cons_price_idx  euribor_three_months
0          0          93.994                 4.857
1          1          93.994                 4.857
2          2          93.994                 4.857
3          3          93.994                 4.857
4          4          93.994                 4.857


2.1 Cleaning the data

2.1.1 cleaning client df data

In [102]:
client['job'] = client['job'].str.replace('.', '_')

client['education'] = client['education'].str.replace('.', '_')
client['education'] = client['education'].replace('unknown', np.NaN)

for col in ['credit_default', 'mortgage']:
    client[col] = client[col].map({
        'yes': 1,
        'no': 0,
        'unknown': 0
    })

print(client.value_counts())

client_id  age  job         marital  education            credit_default  mortgage
0          56   housemaid   married  basic_4y             0               0           1
27443      31   technician  married  university_degree    0               1           1
27435      43   technician  single   high_school          0               0           1
27436      55   technician  single   professional_course  0               0           1
27437      37   technician  married  professional_course  0               1           1
                                                                                     ..
13796      48   technician  married  basic_6y             0               0           1
13797      28   admin_      single   university_degree    0               1           1
13798      32   technician  married  high_school          0               0           1
13799      31   technician  married  professional_course  0               0           1
41187      74   retired     married  

2.1.2 cleaning campaign df data 

In [103]:
campaign['previous_outcome'] = campaign['previous_outcome'].map({'success': 1,
                                                                 'failure': 0,
                                                                 'nonexistent': 0})

campaign['campaign_outcome'] = campaign['campaign_outcome'].map({'yes': 1, 'no': 0})

2.2 Creating new columns

2.2.1 creating year column

In [104]:
df['year'] = 2022

2.2.2 creating last_contact_date column

In [105]:
last_contact_date = pd.to_datetime(df[['year', 'month', 'day']].astype(str).agg('-'.join, axis=1), infer_datetime_format=True, errors='coerce')

campaign['last_contact_date'] = last_contact_date

2.3 Converting data types

2.3.1 Converting data types of client columns

In [106]:
print(client.dtypes)

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default     int64
mortgage           int64
dtype: object


In [107]:
client['credit_default'] = client['credit_default'].astype('bool')

client['mortgage'] = client['mortgage'].astype('bool')

print(client.dtypes)

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default      bool
mortgage            bool
dtype: object


2.3.2 Converting data types of campaign columns

In [108]:
print(campaign.dtypes)

client_id                              int64
number_contacts                        int64
contact_duration                       int64
previous_campaign_contacts             int64
previous_outcome                       int64
campaign_outcome                       int64
last_contact_date             datetime64[ns]
dtype: object


In [109]:
campaign['previous_outcome'] = campaign['previous_outcome'].astype('bool')

campaign['campaign_outcome'] = campaign['campaign_outcome'].astype('bool')

print(campaign.dtypes)

client_id                              int64
number_contacts                        int64
contact_duration                       int64
previous_campaign_contacts             int64
previous_outcome                        bool
campaign_outcome                        bool
last_contact_date             datetime64[ns]
dtype: object


3.1 Saving data to csv's

In [110]:
client.to_csv('client.csv', index=False)

campaign.to_csv('campaign.csv', index=False)

economics.to_csv('economics.csv', index=False)